# Load data

In [1]:
import pandas as pd
import numpy as np

## Load positive and negative words

In [2]:
neg_words = pd.DataFrame(
    open("../data/negative-words.txt", encoding="utf-8").read().splitlines(),
    columns=["word"]
)

pos_words = pd.DataFrame(
    open("../data/negative-words.txt", encoding="utf-8").read().splitlines(),
    columns=["word"]
)

neg_words["label"] = 0
pos_words["label"] = 1


word_df = pd.concat([pos_words, neg_words], ignore_index=True)

#shuffle rows
word_df = word_df.sample(frac=1).reset_index(drop=True)

word_df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xef in position 31529: invalid continuation byte

## Load positive and negative reviews

In [ ]:
neg_reviews = pd.DataFrame(
    open("../data/negative-reviews.txt", encoding="utf-8").read().splitlines(),
    columns=["review"]
)

pos_reviews = pd.DataFrame(
    open("../data/positive-reviews.txt", encoding="utf-8").read().splitlines(),
    columns=["review"]
)

neg_reviews["label"] = 0
pos_reviews["label"] = 1


review_df = pd.concat([neg_reviews, pos_reviews], ignore_index=True)

#shuffle rows
review_df = review_df.sample(frac=1).reset_index(drop=True)

# Feature Selection

## Count Positive and Negative words

In [ ]:
pos_set = set(pos_words["word"])
neg_set = set(neg_words["word"])


review_df["pos_counts"] = review_df["review"].apply(
    lambda text: sum(word.lower() in pos_set for word in text.split())
)

review_df["neg_counts"] = review_df["review"].apply(
    lambda text: sum(word.lower() in neg_set for word in text.split())
)

### Positive counts > 0

In [ ]:
review_df[review_df["pos_counts"] > 0]

,review,label,pos_counts,neg_counts
1,"Flash kills batteries, bad with low light, no ...",0,2,2
6,Epson name is respected; inexpensive photo qua...,1,1,1
8,"Poor reception, placement of IR port.",0,1,1
9,"Poor Color print quality, very noisy, poor cus...",0,2,2
17,"No optical zoom, MPEG Movie feature worthless",0,1,1
...,...,...,...,...
39969,delay between when u press the button and when...,0,1,1
39976,"Bad keypad ergonomics, Bluetooth functions cou...",0,2,2
39982,Good Printer when it works poor service,1,1,1
39987,"flimsy card door,battery door is tricky, compa...",0,1,1


### Negative counts > 0

In [ ]:
review_df[review_df["neg_counts"] > 0]

,review,label,pos_counts,neg_counts
1,"Flash kills batteries, bad with low light, no ...",0,2,2
6,Epson name is respected; inexpensive photo qua...,1,1,1
8,"Poor reception, placement of IR port.",0,1,1
9,"Poor Color print quality, very noisy, poor cus...",0,2,2
17,"No optical zoom, MPEG Movie feature worthless",0,1,1
...,...,...,...,...
39969,delay between when u press the button and when...,0,1,1
39976,"Bad keypad ergonomics, Bluetooth functions cou...",0,2,2
39982,Good Printer when it works poor service,1,1,1
39987,"flimsy card door,battery door is tricky, compa...",0,1,1


## Count number of "no"

In [ ]:
review_df["no_counts"] = review_df["review"].apply(
    lambda text: sum(word.lower() == 'no' for word in text.split())
)

review_df[review_df["no_counts"] > 0]

,review,label,pos_counts,neg_counts,no_counts
1,"Flash kills batteries, bad with low light, no ...",0,2,2,1
14,no USB port (only serial) for downloads,0,0,0,1
17,"No optical zoom, MPEG Movie feature worthless",0,1,1,1
26,Finicky dialing (voice/redial/bluetooth) that ...,0,1,1,1
64,"poor menu system, no alarm feature",0,2,2,1
...,...,...,...,...,...
39924,A complete system all in one. No need to buy m...,1,0,0,1
39928,No Internet capabilities. Size.,0,0,0,1
39946,"Compact and cheap, no paper jams, easy to use,...",1,0,0,1
39968,No optical viewfinder. No image stabilization...,0,0,0,2


## Contain "!"

In [ ]:
review_df["has_exclamation"] = review_df["review"].apply(
    lambda text: 1 if '!' in text else 0
)

review_df[review_df["has_exclamation"] > 0]

,review,label,pos_counts,neg_counts,no_counts,has_exclamation
41,"inexpensive Clik! media, 1.5 MP resolution, 2x...",1,0,0,0,1
57,"Terrible phone! Bad battery life, doesn't noti...",0,2,2,0,1
69,Superb Black and Color Printing! Excellent Ph...,1,0,0,0,1
71,Hey! The calculator works!,1,0,0,0,1
73,none that I can think of!,0,0,0,0,1
...,...,...,...,...,...,...
39938,Easy storage and easy to manuver. VERY durable!!,1,0,0,0,1
39945,"Similar featuresd as 35 MM counterparts, great...",1,0,0,0,1
39955,Sony Support is Horrible at best!,0,1,1,0,1
39966,"a little difficult to see letters on keys, tha...",0,2,2,0,1


## Count Pronouns

In [ ]:
def count_pronoun(text):
    pronouns = ['i', 'me', 'my', 'you', 'your', 'we', 'our', 'us']
    total = 0
    words = text.split()
    for w in words:
        for p in pronouns:
            if p == w.lower():
                total += 1
    return total
    
review_df["pronoun_counts"] = review_df["review"].apply(
    count_pronoun
)

review_df[review_df["pronoun_counts"] > 0]

,review,label,pos_counts,neg_counts,no_counts,has_exclamation,pronoun_counts
30,"When you turn on the camera, you need to push ...",0,0,0,0,0,3
73,none that I can think of!,0,0,0,0,1,1
78,sound quality good enough for me,1,0,0,0,0,1
84,I have yet to find anything this camera does n...,1,0,0,0,0,1
87,"After scan, part that saves, and lets you se...",0,0,0,0,0,1
...,...,...,...,...,...,...,...
39905,Nice feel in my hands and very durable.,1,0,0,0,0,1
39906,"Fits right into your pocket, external caller I...",1,0,0,0,0,1
39933,The ease and simplicity of getting pictures to...,1,0,0,0,0,1
39937,Has almost every feature you could dream of.,1,0,0,0,0,1


## Get log(len(review))

In [ ]:
review_df["log2_review"] = review_df["review"].apply(
    lambda text: np.log2(len(text) or 1)
)

review_df[review_df["log2_review"] > 0]

,review,label,pos_counts,neg_counts,no_counts,has_exclamation,pronoun_counts,log2_review
0,Price,1,0,0,0,0,0,2.321928
1,"Flash kills batteries, bad with low light, no ...",0,2,2,1,0,0,5.832890
2,"Very easy to use, Stores large number of shots",1,0,0,0,0,0,5.523562
3,Not very good for being on a network and used ...,0,0,0,0,0,0,6.087463
4,none,0,0,0,0,0,0,2.000000
...,...,...,...,...,...,...,...,...
39995,"clear text, never jams",1,0,0,0,0,0,4.459432
39996,"need to switch cartridges, no paper catch tray...",0,0,0,1,0,0,5.954196
39997,"Moive Mode, 3x Zoom",1,0,0,0,0,0,4.247928
39998,picture quality,0,0,0,0,0,0,3.906891


# Train the model

## Use LinearSVC from sklearn to build our SVM classification model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer

features = ['review', 'pos_counts', 'neg_counts', 'no_counts', 'has_exclamation', 'pronoun_counts', 'log2_review']
text_features = 'review'
numeric_features = [
    'pos_counts',
    'neg_counts',
    'no_counts',
    'has_exclamation',
    'pronoun_counts',
    'log2_review'
]

X = review_df[features]
y = review_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train:\n", X_train)
print("y_train:\n", y_train)
print("X_test:\n", X_test)
print("y_test:\n", y_test)

preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), text_features),
        ('num', 'passthrough', numeric_features)
    ]
)

model = Pipeline([
    ('preprocess', preprocessor),
    ('svm', LinearSVC())
])

model.fit(X_train, y_train)


X_train:
                                                   review  pos_counts  \
14307                   Worst purchase mistake in memory           2   
17812  Voice dial, stylish appearance, personalized r...           0   
11020  Small, Quality pics, tastes great and is less ...           0   
15158       Small, cool looking, vibrates, easy menu use           0   
24990                          Price tag, only dual mode           0   
...                                                  ...         ...   
6265                    clarity, ease of use, lcd, flash           0   
11284                         Printing Speed, Ink hunger           0   
38158                                        Good camera           0   
860                           Cheap, fast, low ink costs           0   
15795  Horrible sound quality, terrible speaker phone...           2   

       neg_counts  no_counts  has_exclamation  pronoun_counts  log2_review  
14307           2          0                0   

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('svm', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('text', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers conta

# Result

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Results")

result_df = pd.DataFrame({
    "X_test": X_test['review'],
    "y_test": y_test,
    "y_pred": y_pred
})

result_df.head()


Accuracy: 0.93625
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3997
           1       0.94      0.93      0.94      4003

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

Results


,X_test,y_test,y_pred
32823,"Easy to use/setup, separate black amp;amp; co...",1,1
16298,LIGHTWEIGHT,1,1
28505,"Power button flies off if you sneeze, Missing/...",0,0
6689,The serial Link is slow. The packaged software...,0,0
26893,the printing is a very clear..,1,1


## Select only the review with no features

In [ ]:
X = review_df['review']
y = review_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train:\n", X_train)
print("y_train:\n", y_train)
print("X_test:\n", X_test)
print("y_test:\n", y_test)

model = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("svm", LinearSVC())
])

model.fit(X_train, y_train)

X_train:
 14307                     Worst purchase mistake in memory
17812    Voice dial, stylish appearance, personalized r...
11020    Small, Quality pics, tastes great and is less ...
15158         Small, cool looking, vibrates, easy menu use
24990                            Price tag, only dual mode
                               ...                        
6265                      clarity, ease of use, lcd, flash
11284                           Printing Speed, Ink hunger
38158                                          Good camera
860                             Cheap, fast, low ink costs
15795    Horrible sound quality, terrible speaker phone...
Name: review, Length: 32000, dtype: object
y_train:
 14307    0
17812    1
11020    1
15158    1
24990    0
        ..
6265     1
11284    0
38158    1
860      1
15795    0
Name: label, Length: 32000, dtype: int64
X_test:
 32823    Easy to use/setup, separate black  amp;amp; co...
16298                                          LIGHTWEIGHT

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('svm', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Results")

result_df = pd.DataFrame({
    "X_test": X_test,
    "y_test": y_test,
    "y_pred": y_pred
})

result_df.head()

Accuracy: 0.936125
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3997
           1       0.94      0.93      0.94      4003

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

Results


,X_test,y_test,y_pred
32823,"Easy to use/setup, separate black amp;amp; co...",1,1
16298,LIGHTWEIGHT,1,1
28505,"Power button flies off if you sneeze, Missing/...",0,0
6689,The serial Link is slow. The packaged software...,0,0
26893,the printing is a very clear..,1,1
